## Архитектуры нейронок для рахных задач

In [ ]:
# 1. Координаты X, Y, TVD1 и TVD2 лежат отдельными листами в .xlsx файле
# 2. Файлов .tfrecord не пугаться, в деталях коммита на всякий указал как открывать такие. 
# Там 4 таких файла уже в порядке следования дат, для нефти, воды, пластового и забойного.
# Каждый тензор размерности [121,6,6]. 
# 3. Чтобы узнать какой дате соответствует индекс в тензоре, можно заюзать файл Dates_processed.csv.
# Загружаете как датафрейм и запрашиваете по индексу строки, получаете дату.

In [ ]:
# imports 

import tensorflow as tf
import os 
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, Dense, LSTM, TimeDistributed, Input, Concatenate
from tensorflow.keras.models import Model

In [122]:
# specify your directory where files are located 
# tensors loading

files_dir = '../data/processed/'
loaded_tensors = dict()
for file in [x for x in os.listdir(files_dir) if x.endswith("tfrecord")]:
    serialized_tensor = tf.io.read_file(os.path.join(files_dir,file))
    loaded_tensors[file] = tf.io.parse_tensor(serialized_tensor, out_type=tf.float32)

# coordinates loading 
df_coords = pd.read_excel(os.path.join(files_dir,"Coordinates_processed.xlsx"),sheet_name=['X','Y','TVD1','TVD2'],header=None)
df_X_Y = pd.DataFrame({col: list(zip(df_coords["X"][col], df_coords["Y"][col])) for col in  df_coords["X"].columns})
df_TVD = pd.DataFrame({col: list(zip(df_coords["TVD1"][col], df_coords["TVD2"][col])) for col in  df_coords["TVD1"].columns})

# time loading 
time_sequence = pd.read_csv(os.path.join(files_dir,"Dates_processed.csv"))

In [ ]:
# Параметры
total_time_steps = 121  # Общее количество временных точек
train_time_steps = 84   # Количество временных точек для обучения (до 2027 года)
test_time_steps = total_time_steps - train_time_steps  # Количество временных точек для тестирования

map_height = 6    # Высота карты (матрицы)
map_width = 6     # Ширина карты (матрицы)
batch_size = 1    # Размер пакета

# Входы для обучающей модели
input_oil_train = Input(shape=(train_time_steps, map_height, map_width, 1), name='input_oil')
input_water_train = Input(shape=(train_time_steps, map_height, map_width, 1), name='input_water')
input_reservoir_pressure_train = Input(shape=(train_time_steps, map_height, map_width, 1), name='input_reservoir_pressure')
input_bottomhole_pressure_train = Input(shape=(train_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure')

# Входы для тестовой модели
input_oil_test = Input(shape=(test_time_steps, map_height, map_width, 1), name='input_oil')
input_water_test = Input(shape=(test_time_steps, map_height, map_width, 1), name='input_water')
input_reservoir_pressure_test = Input(shape=(test_time_steps, map_height, map_width, 1), name='input_reservoir_pressure')
input_bottomhole_pressure_test = Input(shape=(test_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure')

# CNN для каждого вида данных
def create_cnn(input_tensor):
    x = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(input_tensor)
    x = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(x)
    x = TimeDistributed(Flatten())(x)
    return x

# Модель для обучения
cnn_oil_train = create_cnn(input_oil_train)
cnn_water_train = create_cnn(input_water_train)
cnn_reservoir_pressure_train = create_cnn(input_reservoir_pressure_train)
cnn_bottomhole_pressure_train = create_cnn(input_bottomhole_pressure_train)

merged_cnn_train = Concatenate(axis=-1)([cnn_oil_train, cnn_water_train, cnn_reservoir_pressure_train, cnn_bottomhole_pressure_train])
lstm_out_train = LSTM(128, return_sequences=False)(merged_cnn_train)
output_train = Dense(1, activation='linear', name='output')(lstm_out_train)

model_train = Model(inputs=[input_oil_train, input_water_train, input_reservoir_pressure_train, input_bottomhole_pressure_train], outputs=output_train)
model_train.compile(optimizer='adam', loss='mse')

# Модель для тестирования
cnn_oil_test = create_cnn(input_oil_test)
cnn_water_test = create_cnn(input_water_test)
cnn_reservoir_pressure_test = create_cnn(input_reservoir_pressure_test)
cnn_bottomhole_pressure_test = create_cnn(input_bottomhole_pressure_test)

merged_cnn_test = Concatenate(axis=-1)([cnn_oil_test, cnn_water_test, cnn_reservoir_pressure_test, cnn_bottomhole_pressure_test])
lstm_out_test = LSTM(128, return_sequences=False)(merged_cnn_test)
output_test = Dense(1, activation='linear', name='output')(lstm_out_test)

model_test = Model(inputs=[input_oil_test, input_water_test, input_reservoir_pressure_test, input_bottomhole_pressure_test], outputs=output_test)
model_test.compile(optimizer='adam', loss='mse')

# Загрузка данных из тензоров
oil_data = np.expand_dims(loaded_tensors['debit_neft_series.tfrecord'].numpy(), axis=-1)
water_data = np.expand_dims(loaded_tensors['debit_water_series.tfrecord'].numpy(), axis=-1)
reservoir_pressure_data = np.expand_dims(loaded_tensors['debit_plast_pressure_series.tfrecord'].numpy(), axis=-1)
bottomhole_pressure_data = np.expand_dims(loaded_tensors['debit_zaboy_pressure_series.tfrecord'].numpy(), axis=-1)

# Проверка форм данных
print("Oil data shape:", oil_data.shape)
print("Water data shape:", water_data.shape)
print("Reservoir pressure data shape:", reservoir_pressure_data.shape)
print("Bottomhole pressure data shape:", bottomhole_pressure_data.shape)

# Убедимся, что формы данных совпадают с ожидаемой формой
oil_data = oil_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
water_data = water_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
reservoir_pressure_data = reservoir_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
bottomhole_pressure_data = bottomhole_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))

# Разделим данные на обучающую и тестовую выборки
oil_train = oil_data[:, :train_time_steps, :, :, :]
water_train = water_data[:, :train_time_steps, :, :, :]
reservoir_pressure_train = reservoir_pressure_data[:, :train_time_steps, :, :, :]
bottomhole_pressure_train = bottomhole_pressure_data[:, :train_time_steps, :, :, :]

oil_test = oil_data[:, train_time_steps:, :, :, :]
water_test = water_data[:, train_time_steps:, :, :, :]
reservoir_pressure_test = reservoir_pressure_data[:, train_time_steps:, :, :, :]
bottomhole_pressure_test = bottomhole_pressure_data[:, train_time_steps:, :, :, :]

# Дебит нефти для обучения и тестирования
output_train = oil_data[:, :train_time_steps, map_height//2, map_width//2, 0].reshape((batch_size, train_time_steps, 1))
output_test = oil_data[:, train_time_steps:, map_height//2, map_width//2, 0].reshape((batch_size, test_time_steps, 1))

# Обучение модели
model_train.fit([oil_train, water_train, reservoir_pressure_train, bottomhole_pressure_train], output_train, epochs=10)

# Оценка модели на тестовых данных
model_test.evaluate([oil_test, water_test, reservoir_pressure_test, bottomhole_pressure_test], output_test)

In [ ]:
oil_data = np.expand_dims(loaded_tensors['debit_neft_series.tfrecord'].numpy(), axis=-1)
water_data = np.expand_dims(loaded_tensors['debit_water_series.tfrecord'].numpy(), axis=-1)
reservoir_pressure_data = np.expand_dims(loaded_tensors['debit_plast_pressure_series.tfrecord'].numpy(), axis=-1)
bottomhole_pressure_data = np.expand_dims(loaded_tensors['debit_zaboy_pressure_series.tfrecord'].numpy(), axis=-1)